# Imports

In [1]:
import os
import logging
import pandas as pd
import datetime
import numpy as np
import subprocess

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
from matplotlib.pyplot import Polygon
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter, MinuteLocator
import matplotlib.ticker as ticker

import ipywidgets as widgets
from ipywidgets import interact

from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, LinearAxis, Range1d
from bokeh.palettes import Spectral10

from IPython.display import Image

try:
    from PT3S import dxAndMxHelperFcts
except:
    import dxAndMxHelperFcts

try:
    from PT3S import Rm
except:
    import Rm

In [2]:
import importlib

In [3]:
importlib.reload(dxAndMxHelperFcts)

<module 'dxAndMxHelperFcts' from 'c:\\users\\wolters\\3s\\pt3s\\dxAndMxHelperFcts.py'>

# Logging

In [4]:
logger = logging.getLogger()  

logFileName= r"Example3.log" 

loglevel = logging.DEBUG
logging.basicConfig(filename=logFileName
                        ,filemode='w'
                        ,level=loglevel
                        ,format="%(asctime)s ; %(name)-60s ; %(levelname)-7s ; %(message)s")    

fileHandler = logging.FileHandler(logFileName)     

logger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logging.Formatter("%(levelname)-7s ; %(message)s"))
consoleHandler.setLevel(logging.INFO)
logger.addHandler(consoleHandler)

# Read Model and Results

In [5]:
dbFilename="DistrictHeating"
dbFile=os.path.join(os.path.dirname(os.path.abspath(dxAndMxHelperFcts.__file__))
                    +'/Examples/'
                    +dbFilename
                    +'.db3'
)

In [6]:
m=dxAndMxHelperFcts.readDxAndMx(dbFile=dbFile                                 
                                ,preventPklDump=True
                                #,maxRecords=-1
)

INFO    ; Dx.__init__: dbFile (abspath): c:\users\wolters\3s\pt3s\Examples\DistrictHeating.db3 exists readable ...
INFO    ; Mx.setResultsToMxsFile: Mxs: ..\Examples\WDDistrictHeating\B1\V0\BZ1\M-1-0-1.1.MXS reading ...
INFO    ; dxWithMx.__init__: DistrictHeating: processing dx and mx ...


In [7]:
m.dx.dataFrames['V3_VBEL'].shape

(883, 572)

In [8]:
m.V3_VBEL.shape#head(100)

(883, 585)

In [9]:
1/0

ZeroDivisionError: division by zero

# Longitudinal Sections

In [10]:
dfAGSN=m.dfAGSN

In [11]:
#dfAGSN

In [12]:
#dfAGSN[dfAGSN['nextNODE'].isin(['V-K1683S'])]

In [13]:
#dfAGSN[dfAGSN['nextNODE'].isin(['V-K1683S'])]

In [14]:
dfAGSN=dxAndMxHelperFcts.constructNewMultiindexFromCols(dfAGSN,mColNames=['TYPE','ID']).sort_values(by=['LFDNR','XL','Pos'])
dfAGSN

Pos                   pk                   tk  \
OBJTYPE OBJID                                                               
ROHR    5691533564979419761   0  5755933101669454049  5755933101669454049   
        5048873293262650113   1  5755933101669454049  5755933101669454049   
        5715081934973525403   2  5755933101669454049  5755933101669454049   
        5413647981880727734   3  5755933101669454049  5755933101669454049   
        5134010500800575790   4  5755933101669454049  5755933101669454049   
...                          ..                  ...                  ...   
        4621030304810285220  39  4868980900521118307  4868980900521118307   
        5046251450517706156  40  4868980900521118307  4868980900521118307   
        5478012163155701568  41  4868980900521118307  4868980900521118307   
        4968402990875758821  42  4868980900521118307  4868980900521118307   
        4814824415166736381  43  4868980900521118307  4868980900521118307   

                            LFDNR                   NAME XL compNr  nextNODE  
OBJTYPE OBJID                                                                 
ROHR    5691533564979419761   1.0           Längsschnitt  0      1  V-K1683S  
        5048873293262650113   1.0           Längsschnitt  0      1  V-K1693S  
        5715081934973525403   1.0           Längsschnitt  0      1  V-K2163S  
        5413647981880727734   1.0           Längsschnitt  0      1  V-K2043S  
        5134010500800575790   1.0           Längsschnitt  0      1  V-K2033S  
...                           ...                    ... ..    ...       ...  
        4621030304810285220   3.0  Längsschnitt in bar,B  2      1  R-K2583S  
        5046251450517706156   3.0  Längsschnitt in bar,B  2      1  R-K2593S  
        5478012163155701568   3.0  Längsschnitt in bar,B  2      1  R-K2603S  
        4968402990875758821   3.0  Längsschnitt in bar,B  2      1  R-K2613S  
        4814824415166736381   3.0  Längsschnitt in bar,B  2      1  R-K2623S  

[528 rows x 8 columns]

In [15]:
colsAGSN=dfAGSN.columns

In [16]:
colsAGSN

Index(['Pos', 'pk', 'tk', 'LFDNR', 'NAME', 'XL', 'compNr', 'nextNODE'], dtype='object')

In [17]:
dfAGSN=pd.merge(dfAGSN,m.V3_VBEL,left_index=True,right_index=True,suffixes=('','_VBEL')).sort_values(by=['LFDNR','XL','Pos'])

In [25]:
t0=pd.Timestamp(m.mx.df.index[0].strftime('%Y-%m-%d %X.%f'))

# "('STAT', 'KNOT~*~*~*~PH', Timestamp('2023-02-12 23:00:00'), Timestamp('2023-02-12 23:00:00'))_i",

PH_i=str(('STAT'
,'KNOT~*~*~*~PH'
,t0
,t0
))+'_i'

PH_k=str(('STAT'
,'KNOT~*~*~*~PH'
,t0
,t0
))+'_k'

In [29]:
dfAGSN.filter(items=colsAGSN.to_list()+['NAME_i','NAME_k',PH_i,PH_k,'QM'])

Pos                   pk                   tk  \
OBJTYPE OBJID                                                               
ROHR    5691533564979419761   0  5755933101669454049  5755933101669454049   
        5048873293262650113   1  5755933101669454049  5755933101669454049   
        5715081934973525403   2  5755933101669454049  5755933101669454049   
        5413647981880727734   3  5755933101669454049  5755933101669454049   
        5134010500800575790   4  5755933101669454049  5755933101669454049   
...                          ..                  ...                  ...   
        4621030304810285220  39  4868980900521118307  4868980900521118307   
        5046251450517706156  40  4868980900521118307  4868980900521118307   
        5478012163155701568  41  4868980900521118307  4868980900521118307   
        4968402990875758821  42  4868980900521118307  4868980900521118307   
        4814824415166736381  43  4868980900521118307  4868980900521118307   

                            LFDNR                   NAME XL compNr  nextNODE  \
OBJTYPE OBJID                                                                  
ROHR    5691533564979419761   1.0           Längsschnitt  0      1  V-K1683S   
        5048873293262650113   1.0           Längsschnitt  0      1  V-K1693S   
        5715081934973525403   1.0           Längsschnitt  0      1  V-K2163S   
        5413647981880727734   1.0           Längsschnitt  0      1  V-K2043S   
        5134010500800575790   1.0           Längsschnitt  0      1  V-K2033S   
...                           ...                    ... ..    ...       ...   
        4621030304810285220   3.0  Längsschnitt in bar,B  2      1  R-K2583S   
        5046251450517706156   3.0  Längsschnitt in bar,B  2      1  R-K2593S   
        5478012163155701568   3.0  Längsschnitt in bar,B  2      1  R-K2603S   
        4968402990875758821   3.0  Längsschnitt in bar,B  2      1  R-K2613S   
        4814824415166736381   3.0  Längsschnitt in bar,B  2      1  R-K2623S   

                               NAME_i    NAME_k  \
OBJTYPE OBJID                                     
ROHR    5691533564979419761      V-E0  V-K1683S   
        5048873293262650113  V-K1683S  V-K1693S   
        5715081934973525403  V-K2163S  V-K1693S   
        5413647981880727734  V-K2043S  V-K2163S   
        5134010500800575790  V-K2033S  V-K2043S   
...                               ...       ...   
        4621030304810285220  R-K2573S  R-K2583S   
        5046251450517706156  R-K2583S  R-K2593S   
        5478012163155701568  R-K2593S  R-K2603S   
        4968402990875758821  R-K2603S  R-K2613S   
        4814824415166736381  R-K2613S  R-K2623S   

                            ('STAT', 'KNOT~*~*~*~PH', Timestamp('2023-02-12 23:00:00'), Timestamp('2023-02-12 23:00:00'))_i  \
OBJTYPE OBJID                                                                                                                 
ROHR    5691533564979419761                                           4.878388                                                
        5048873293262650113                                           4.800117                                                
        5715081934973525403                                           4.698539                                                
        5413647981880727734                                           4.629901                                                
        5134010500800575790                                           4.579439                                                
...                                                                        ...                                                
        4621030304810285220                                           1.208033                                                
        5046251450517706156                                           1.196312                                                
        5478012163155701568                        

In [24]:
dfAGSN[PH_i]

OBJTYPE  OBJID              
ROHR     5691533564979419761    4.878388
         5048873293262650113    4.800117
         5715081934973525403    4.698539
         5413647981880727734    4.629901
         5134010500800575790    4.579439
                                  ...   
         4621030304810285220    1.208033
         5046251450517706156    1.196312
         5478012163155701568     1.09261
         4968402990875758821    1.018315
         4814824415166736381    0.911956
Name: ('STAT', 'KNOT~*~*~*~PH', Timestamp('2023-02-12 23:00:00'), Timestamp('2023-02-12 23:00:00'))_i, Length: 528, dtype: object

In [18]:
dfAGSN.columns.to_list()

['Pos',
 'pk',
 'tk',
 'LFDNR',
 'NAME',
 'XL',
 'compNr',
 'nextNODE',
 'pk_VBEL',
 'fkDE',
 'rk',
 'fkKI',
 'BESCHREIBUNG',
 'DGR',
 'DKL',
 'ALPHA',
 'VGREST',
 'QLBMAX',
 'QLEKL',
 'RGBEVE',
 'TROHR',
 'RKPBEV',
 'POEFF',
 'IBEDEF',
 'IBETYP',
 'KNOTK',
 'ZKOR',
 'fkFSTF',
 'IEKL',
 'fkCONT',
 'GEOMWKB',
 'DELETED',
 'SELECT1',
 'IDREFERENZ',
 'KENNUNG',
 'IPLANUNG',
 'pk_BV',
 'fkDE_BV',
 'fk',
 'GEOM_CNLN_I',
 'GRAF',
 'GRAF_CNLN_I',
 'MId',
 'MText',
 'Basis',
 'Variante',
 'BZ',
 'Geaendert',
 'Erstellt',
 'pk_VMBASIS',
 'fkBASIS',
 'fkVARIANTE',
 'fkBZ',
 'MZ',
 'MT',
 'MId_VMVARIANTE',
 'MText_VMVARIANTE',
 'Basis_VMVARIANTE',
 'Variante_VMVARIANTE',
 'BZ_VMVARIANTE',
 'Geaendert_VMVARIANTE',
 'Erstellt_VMVARIANTE',
 'pk_VMVARIANTE',
 'fkBASIS_VMVARIANTE',
 'fkVARIANTE_VMVARIANTE',
 'fkBZ_VMVARIANTE',
 'MZ_VMVARIANTE',
 'MT_VMVARIANTE',
 'pk_CONT',
 'fkDE_CONT',
 'rk_CONT',
 'tk_CONT',
 'ID_CONT',
 'NAME_CONT',
 'IDPARENT_CONT',
 'rkPARENT_CONT',
 'LFDNR_CONT',
 'GRAF_CONT',


In [ ]:
1/0

## Get Nodes

In [ ]:
dfAGSN=m.dfAGSN

In [ ]:
dfAGSN

In [ ]:
dfAGSN=dfAGSN[['ID','Pos', 'LFDNR']]

In [ ]:
dfVBEL=m.V3_VBEL.reset_index()

In [ ]:
dfVBEL=dfVBEL[['OBJID','pk_i', 'pk_k']]

In [ ]:
df=pd.merge(dfAGSN, dfVBEL, left_on='ID', right_on='OBJID', how='inner')

In [ ]:
df=df.drop_duplicates()

In [ ]:
df_i=df[df['pk_i'].notna()]
df_k=df[df['pk_k'].notna()]
df_i=df_i.rename(columns={'pk_i': 'pk'})
df_k=df_k.rename(columns={'pk_k': 'pk'})
df=pd.concat([df_i, df_k])

In [ ]:
df=df.drop(columns=['pk_i', 'pk_k', 'OBJID'])

In [ ]:
df=df.sort_values('ID')
df=df.drop_duplicates(subset=['Pos', 'LFDNR', 'pk'], keep='first')

In [ ]:
df=df.sort_values(['Pos', 'LFDNR'])

## Get Node Results

In [ ]:
dfKNOT=m.V3_KNOT

In [ ]:
df=pd.merge(df, dfKNOT, on='pk', how='left')

In [ ]:
df=df[~df['NAME'].str.startswith('R')]

In [ ]:
new_names = {df.columns[162]: 'P', df.columns[194]: 'QM', df.columns[202]: 'T'}#!!!!!!!Massenstrom nicht Volumenstrom
df.rename(columns=new_names, inplace=True)

In [ ]:
cols_to_keep=['ID', 'Pos', 'LFDNR', 'pk', 'NAME','P','QM','T']

In [ ]:
df=df[cols_to_keep]

In [ ]:
dfs = [df[df['LFDNR'] == i] for i in [1.0, 2.0, 3.0]]

In [ ]:
for df in dfs:
    df.reset_index(inplace=True, drop=True)

In [ ]:
dfs[0].head()

# Plot

## Define Axes

In [ ]:
def fyP(ax,offset=0):
    ax.spines["left"].set_position(("outward", offset))
    ax.set_ylabel('Druck in bar')
    ax.set_ylim(1,6)
    ax.set_yticks(sorted(np.append(np.linspace(1,6,11),[])))
    ax.yaxis.set_ticks_position('left')
    ax.yaxis.set_label_position('left')

def fyM(ax,offset=60):
    Rm.pltLDSHelperY(ax)
    ax.spines["left"].set_position(("outward",offset))
    ax.set_ylabel('Massenstrom in t/h')
    ax.set_ylim(500,550)
    ax.set_yticks(sorted(np.append(np.linspace(500,550,11),[])))
    ax.yaxis.set_ticks_position('left')
    ax.yaxis.set_label_position('left')
    
def fyT(ax,offset=120):
    Rm.pltLDSHelperY(ax)
    ax.spines["left"].set_position(("outward",offset))
    ax.set_ylabel('Tempertatur in °C')
    ax.set_ylim(85,95)
    ax.set_yticks(sorted(np.append(np.linspace(0,95,11),[])))
    ax.yaxis.set_ticks_position('left')
    ax.yaxis.set_label_position('left')

## Non-interactive Plot

In [ ]:
def plot():
                       
    fig, ax0 = plt.subplots(figsize=Rm.DINA3q)
    
    ax0.set_yticks(np.linspace(0, 10, 21))
    ax0.yaxis.set_ticklabels([])
    ax0.grid()

    #P
    ax1 = ax0.twinx()     
    fyP(ax1) 
    ax1.plot(df.index, df['P'], color='red', label='P')
    
    #M
    ax2 = ax0.twinx()     
    fyM(ax2)
    ax2.plot(df.index, df['QM'], color='orange', label='T')
    
    #T
    ax3 = ax0.twinx() 
    fyT(ax3)
    ax3.plot(df.index, df['T'], color='pink', label='M')
     
    # Set the x-axis limits
    ax0.set_xlim(0, 87)

    #Rm.pltHelperX(ax0, dateFormat='%Hh:%Mm:%Ss', bysecond=list(range(0, 61, 2)), yPos=0)
    
    ax0.set_title('Longitudinal Section for '+dbFilename)

    #Create printable Output
    plt.savefig('Example3_Output.pdf', format='pdf', dpi=300)
    plt.savefig('Example2_Output.png', format='png', dpi=300)
    
    plt.show()

In [ ]:
plot()

In [ ]:
def f():
    try:
        pass
        1/0
    except Exception as e:
        print("caught!")
        raise

In [ ]:
f()

In [ ]:
try:
    f()
except Exception as e:
    print("caught II!")
    raise

In [ ]:
pt3s_path = os.path.dirname(os.path.abspath(dxAndMxHelperFcts.__file__))

In [ ]:
drive,dummy=os.path.splitdrive(pt3s_path)

In [ ]:
pt3s_path

In [ ]:
import glob
glob.glob(os.path.join(drive,'*.exe'))

In [ ]:
import os
from glob import glob

files = []
start_dir = r"C:\\3S" #drive#os.getcwd()
pattern   = "SirCalc.exe"

for file,_,_ in os.walk(start_dir):
    files.extend(glob(os.path.join(file,pattern))) 
    
files

In [ ]:
[f  for f in reversed(sorted(files,key=lambda file: os.path.getmtime(file)) )]